* [Home - Keras Documentation](https://keras.io/ja/)
* [kerasのmnistのサンプルを読んでみる - Qiita](https://qiita.com/ash8h/items/29e24fc617b832fba136)

# ライブラリのインポート

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

# 学習曲線描画
# pip install livelossplot
from livelossplot import PlotLossesKeras

# 共通変数定義

In [ ]:
batch_size = 128
num_classes = 10
epochs = 20

# MNISTデータセット取得

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# テストデータの整形

keras.datasets.mnist.load_data()で取得するMNIST画像データは28px X 28px、0〜255の８ビットグレースケールを元にしたデータになっているため、下記の整形を行います。

* 行列の次元変換： (28, 28) → (784)
* 画素値の正規化： 0〜255 → 0〜1の間の小数

In [ ]:
x_train = x_train.reshape(60000, 784) # 2次元配列を1次元に変換
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')   # int型をfloat32型に変換
x_test = x_test.astype('float32')
x_train /= 255                        # [0-255]の値を[0.0-1.0]に変換
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# モデル構築

* [ソフトマックス関数](https://mathtrain.jp/softmax)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='softmax', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# モデル構造の可視化

[可視化 - Keras Documentation](https://keras.io/ja/visualization/)

In [ ]:
# PNG形式でモデルの構造を出力する
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png')

# SVG形式でモデルの構造を出力する
# 環境によっては''''pip install pydot''を実行し、pydotをインストールする必要あり
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

# モデルの最適化

In [ ]:
stack = model.fit(x_train, y_train,  # 画像とラベルデータ
                    batch_size=batch_size,
                    epochs=epochs,     # エポック数の指定
                    verbose=1,         # ログ出力の指定. 0だとログが出ない
                    validation_data=(x_test, y_test),
                    callbacks=[PlotLossesKeras()])

# AccuracyとLossの可視化

In [ ]:
import matplotlib.pyplot as plt

x = range(epochs)
plt.plot(x, stack.history['acc'], label="acc")
plt.title("accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.plot(x, stack.history['loss'], label="loss")
plt.title("loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

# 評価

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# WeightとBiasの確認

* [Kerasからパラメータ抽出についてのメモ](https://qiita.com/Y_Aowashi/items/2c452d9cdbc638ea5763)
* [kerasのmodel.get_weights()で得られるlistの構造](https://qiita.com/shyne/items/31721724820dcfae4baf)

In [ ]:
# numpy
import numpy as np

layer = model.layers[0]        # 隠れ層なしなのでlayer[0]
w = layer.get_weights()[0]
w = np.array(w)
print('w.shape: ', w.shape)

wt = w.transpose()
print('wt.shape: ', wt.shape)

b = layer.get_weights()[1]
b = np.array(b)
print('b.shape: ', b.shape)

In [ ]:
# 画像表示
import matplotlib.cm as cm
from matplotlib import pylab as plt

fig, axarr = plt.subplots(2, 5)
for idx in range(10):
    ax = axarr[int(idx / 5)][idx % 5]
    img_src = (wt[idx, :] * 100).astype(np.int32)

    #print((weights[idx, :].array * 100).astype(np.int32))
    ax.imshow(img_src.reshape(28, 28), cmap = cm.Greys_r)
    ax.set_title(str(idx))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
plt.show()
plt.close()

In [ ]:
print(b)